# Module 7.1: Advanced Defect Detection

**Learning Objectives:**
- Understand object detection for semiconductor defect identification
- Compare classical computer vision vs deep learning approaches
- Implement YOLO, Faster R-CNN, and classical detection pipelines
- Evaluate models using manufacturing-specific metrics
- Deploy production-ready defect detection systems

**Prerequisites:** Basic computer vision, neural networks, Python programming

In [ ]:
# Setup and imports
import sys
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Add current directory to path for local imports
sys.path.insert(0, '.')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import json
import time
from typing import List, Dict, Any, Tuple

# Data directory path resolution
DATA_DIR = Path('../../../datasets').resolve()
print(f"Data directory: {DATA_DIR}")

# Configure matplotlib
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 11

print("Setup complete!")

## 1. Understanding Defect Detection

Defect detection in semiconductor manufacturing involves identifying and localizing anomalies on wafer surfaces. Common defects include:

- **Scratches**: Linear defects from mechanical damage
- **Particles**: Foreign matter contamination  
- **Cracks**: Fracture patterns in the material

Let's start by importing our pipeline components:

In [ ]:
# Import our detection pipeline
try:
    from src.advanced_defect_detection_pipeline import (
        AdvancedDefectDetectionPipeline,
        generate_synthetic_wafer_defects,
        ClassicalDetector,
        calculate_iou,
        evaluate_detection_metrics
    )
except ImportError:
    # Direct import from pipeline file
    import importlib.util
    spec = importlib.util.spec_from_file_location(
        "pipeline", "7.1-advanced-defect-detection-pipeline.py"
    )
    sys.modules["pipeline"] = importlib.util.module_from_spec(spec)
    pipeline = sys.modules["pipeline"]
    spec.loader.exec_module(pipeline)
    
    AdvancedDefectDetectionPipeline = pipeline.AdvancedDefectDetectionPipeline
    generate_synthetic_wafer_defects = pipeline.generate_synthetic_wafer_defects
    ClassicalDetector = pipeline.ClassicalDetector
    calculate_iou = pipeline.calculate_iou
    evaluate_detection_metrics = pipeline.evaluate_detection_metrics

print("Pipeline imported successfully!")

## 2. Synthetic Wafer Data Generation

Since real wafer defect data is proprietary, we'll generate synthetic wafer images with known defects for learning purposes.

In [ ]:
# Generate synthetic wafer data
print("Generating synthetic wafer defect data...")
images, annotations = generate_synthetic_wafer_defects(
    n_images=10,
    image_size=(400, 400),
    n_defects_range=(2, 5),
    seed=42
)

print(f"Generated {len(images)} images with annotations")
print(f"Image shape: {images[0].shape}")
print(f"Example annotations: {len(annotations[0])} defects in first image")

In [ ]:
# Visualize synthetic wafer images with defects
def draw_bounding_boxes(image, annotations, title="Defect Detection"):
    """Draw bounding boxes on image."""
    img_with_boxes = image.copy()
    colors = {'scratch': (0, 255, 0), 'particle': (255, 0, 0), 'crack': (0, 0, 255)}
    
    for ann in annotations:
        bbox = ann['bbox']
        class_name = ann['class']
        color = colors.get(class_name, (255, 255, 255))
        
        cv2.rectangle(img_with_boxes, 
                     (int(bbox[0]), int(bbox[1])), 
                     (int(bbox[2]), int(bbox[3])), 
                     color, 2)
        
        # Add label
        cv2.putText(img_with_boxes, class_name, 
                   (int(bbox[0]), int(bbox[1]-5)), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
    
    return img_with_boxes

# Display first few images with annotations
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for i in range(6):
    img_with_boxes = draw_bounding_boxes(images[i], annotations[i])
    axes[i].imshow(cv2.cvtColor(img_with_boxes, cv2.COLOR_BGR2RGB))
    axes[i].set_title(f"Wafer {i+1}: {len(annotations[i])} defects")
    axes[i].axis('off')

plt.tight_layout()
plt.suptitle("Synthetic Wafer Images with Ground Truth Defects", y=1.02, fontsize=16)
plt.show()

# Print defect statistics
defect_counts = {'scratch': 0, 'particle': 0, 'crack': 0}
for ann_list in annotations:
    for ann in ann_list:
        defect_counts[ann['class']] += 1

print("\nDefect Distribution:")
for defect_type, count in defect_counts.items():
    print(f"{defect_type.capitalize()}: {count}")

## 3. Classical Computer Vision Approach

Let's start with a classical approach using OpenCV for defect detection. This method uses traditional image processing techniques without requiring training data.

In [ ]:
# Initialize classical detector
classical_pipeline = AdvancedDefectDetectionPipeline(
    backend='classical',
    blur_kernel=5,
    threshold_value=50
)

print(f"Classical pipeline backend: {classical_pipeline.backend}")
print(f"Detector type: {type(classical_pipeline.detector).__name__}")

# Fit the model (no-op for classical)
classical_pipeline.fit(images, annotations)

print("\nClassical pipeline trained (no training required)")

In [ ]:
# Make predictions with classical method
print("Making predictions with classical detector...")
start_time = time.time()
classical_predictions = classical_pipeline.predict(images[:5])
classical_inference_time = time.time() - start_time

print(f"Classical inference time: {classical_inference_time:.3f}s for 5 images")
print(f"Average time per image: {classical_inference_time/5:.3f}s")

# Visualize classical predictions
fig, axes = plt.subplots(2, 5, figsize=(20, 8))

for i in range(5):
    # Ground truth
    gt_img = draw_bounding_boxes(images[i], annotations[i])
    axes[0, i].imshow(cv2.cvtColor(gt_img, cv2.COLOR_BGR2RGB))
    axes[0, i].set_title(f"Ground Truth ({len(annotations[i])} defects)")
    axes[0, i].axis('off')
    
    # Predictions
    pred_img = draw_bounding_boxes(images[i], classical_predictions[i])
    axes[1, i].imshow(cv2.cvtColor(pred_img, cv2.COLOR_BGR2RGB))
    axes[1, i].set_title(f"Classical Predictions ({len(classical_predictions[i])} defects)")
    axes[1, i].axis('off')

plt.tight_layout()
plt.suptitle("Classical Detection Results", y=1.02, fontsize=16)
plt.show()

In [ ]:
# Evaluate classical performance
classical_metrics = classical_pipeline.evaluate(
    images[:5], 
    annotations[:5], 
    iou_threshold=0.5
)

print("Classical Detection Performance:")
print(f"Precision: {classical_metrics['precision']:.3f}")
print(f"Recall: {classical_metrics['recall']:.3f}")
print(f"F1-Score: {classical_metrics['f1_score']:.3f}")
print(f"mAP@0.5: {classical_metrics['map_50']:.3f}")
print(f"PWS: {classical_metrics['pws_percent']:.1f}%")
print(f"Estimated Loss: ${classical_metrics['estimated_loss_usd']:.0f}")
print(f"True Positives: {classical_metrics['true_positives']}")
print(f"False Positives: {classical_metrics['false_positives']}")
print(f"False Negatives: {classical_metrics['false_negatives']}")

## 4. Deep Learning Approaches

Now let's explore deep learning approaches. We'll check for available backends and use them if possible, otherwise fall back gracefully.

In [ ]:
# Check available backends
backend_availability = {}

# Check YOLO (ultralytics)
try:
    import ultralytics
    backend_availability['yolo'] = True
    print("✓ YOLO (ultralytics) available")
except ImportError:
    backend_availability['yolo'] = False
    print("✗ YOLO (ultralytics) not available")

# Check Faster R-CNN (torchvision)
try:
    import torch
    import torchvision
    backend_availability['fasterrcnn'] = True
    print("✓ Faster R-CNN (torchvision) available")
except ImportError:
    backend_availability['fasterrcnn'] = False
    print("✗ Faster R-CNN (torchvision) not available")

backend_availability['classical'] = True
print("✓ Classical (OpenCV) always available")

print(f"\nAvailable backends: {[k for k, v in backend_availability.items() if v]}")

In [ ]:
# Compare different backends
backend_results = {}
inference_times = {}

# Test each available backend
for backend_name in ['classical', 'yolo', 'fasterrcnn']:
    if backend_name == 'classical' or backend_availability.get(backend_name, False):
        print(f"\nTesting {backend_name.upper()} backend...")
        
        try:
            # Initialize pipeline
            pipeline = AdvancedDefectDetectionPipeline(backend=backend_name)
            print(f"Actual backend used: {pipeline.backend}")
            
            # Train (quick for demo)
            pipeline.fit(images[:3], annotations[:3], epochs=1)
            
            # Evaluate
            start_time = time.time()
            metrics = pipeline.evaluate(images[3:6], annotations[3:6])
            inference_time = time.time() - start_time
            
            backend_results[pipeline.backend] = metrics
            inference_times[pipeline.backend] = inference_time
            
            print(f"✓ {pipeline.backend} completed successfully")
            print(f"  Precision: {metrics['precision']:.3f}")
            print(f"  Recall: {metrics['recall']:.3f}")
            print(f"  mAP@0.5: {metrics['map_50']:.3f}")
            print(f"  Inference time: {inference_time:.3f}s")
            
        except Exception as e:
            print(f"✗ {backend_name} failed: {e}")
    else:
        print(f"\nSkipping {backend_name.upper()} backend (not available)")

In [ ]:
# Visualize backend comparison
if len(backend_results) > 1:
    # Create comparison dataframe
    comparison_df = pd.DataFrame(backend_results).T
    comparison_df['inference_time'] = pd.Series(inference_times)
    
    print("\nBackend Comparison:")
    display_cols = ['precision', 'recall', 'f1_score', 'map_50', 'pws_percent', 'inference_time']
    print(comparison_df[display_cols].round(3))
    
    # Plot comparison
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    # Accuracy metrics
    metrics_to_plot = ['precision', 'recall', 'f1_score', 'map_50']
    x_pos = np.arange(len(comparison_df))
    width = 0.2
    
    for i, metric in enumerate(metrics_to_plot):
        axes[0].bar(x_pos + i*width, comparison_df[metric], width, 
                   label=metric.replace('_', ' ').title())
    
    axes[0].set_xlabel('Backend')
    axes[0].set_ylabel('Score')
    axes[0].set_title('Detection Accuracy Metrics')
    axes[0].set_xticks(x_pos + width * 1.5)
    axes[0].set_xticklabels(comparison_df.index)
    axes[0].legend()
    axes[0].set_ylim(0, 1)
    
    # PWS percentage
    axes[1].bar(comparison_df.index, comparison_df['pws_percent'])
    axes[1].set_ylabel('PWS (%)')
    axes[1].set_title('Prediction Within Spec')
    axes[1].set_ylim(0, 100)
    
    # Inference time
    axes[2].bar(comparison_df.index, comparison_df['inference_time'])
    axes[2].set_ylabel('Time (seconds)')
    axes[2].set_title('Inference Time')
    
    plt.tight_layout()
    plt.show()
else:
    print("\nOnly one backend available for comparison.")

## 5. Model Evaluation and Analysis

Let's dive deeper into model evaluation with confusion matrices and detailed error analysis.

In [ ]:
# Detailed error analysis
def analyze_detection_errors(predictions, ground_truth, iou_threshold=0.5):
    """Analyze detection errors in detail."""
    total_gt = sum(len(gt) for gt in ground_truth)
    total_pred = sum(len(pred) for pred in predictions)
    
    class_stats = {'scratch': {'tp': 0, 'fp': 0, 'fn': 0},
                  'particle': {'tp': 0, 'fp': 0, 'fn': 0},
                  'crack': {'tp': 0, 'fp': 0, 'fn': 0}}
    
    for pred_list, gt_list in zip(predictions, ground_truth):
        gt_matched = [False] * len(gt_list)
        
        # Match predictions to ground truth
        for pred in pred_list:
            pred_bbox = pred['bbox']
            pred_class = pred['class']
            best_iou = 0.0
            best_gt_idx = -1
            
            for gt_idx, gt in enumerate(gt_list):
                if gt_matched[gt_idx]:
                    continue
                
                gt_bbox = gt['bbox']
                gt_class = gt['class']
                iou = calculate_iou(pred_bbox, gt_bbox)
                
                if iou > best_iou and pred_class == gt_class:
                    best_iou = iou
                    best_gt_idx = gt_idx
            
            if best_iou >= iou_threshold and best_gt_idx >= 0:
                class_stats[pred_class]['tp'] += 1
                gt_matched[best_gt_idx] = True
            else:
                class_stats[pred_class]['fp'] += 1
        
        # Count missed detections
        for gt_idx, gt in enumerate(gt_list):
            if not gt_matched[gt_idx]:
                class_stats[gt['class']]['fn'] += 1
    
    return class_stats, total_gt, total_pred

# Analyze classical detector errors
class_stats, total_gt, total_pred = analyze_detection_errors(
    classical_predictions, annotations[:5]
)

print("\nPer-Class Performance Analysis:")
print(f"{'Class':<10} {'Precision':<10} {'Recall':<10} {'F1-Score':<10}")
print("-" * 45)

for class_name, stats in class_stats.items():
    tp, fp, fn = stats['tp'], stats['fp'], stats['fn']
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0.0
    
    print(f"{class_name:<10} {precision:<10.3f} {recall:<10.3f} {f1:<10.3f}")

print(f"\nTotal Ground Truth Objects: {total_gt}")
print(f"Total Predicted Objects: {total_pred}")

In [ ]:
# IoU threshold sensitivity analysis
iou_thresholds = [0.3, 0.5, 0.7, 0.9]
threshold_results = []

print("IoU Threshold Sensitivity Analysis:")
print(f"{'IoU Threshold':<15} {'Precision':<10} {'Recall':<10} {'F1-Score':<10}")
print("-" * 50)

for threshold in iou_thresholds:
    metrics = evaluate_detection_metrics(
        classical_predictions, annotations[:5], iou_threshold=threshold
    )
    
    threshold_results.append({
        'threshold': threshold,
        'precision': metrics['precision'],
        'recall': metrics['recall'],
        'f1_score': metrics['f1_score']
    })
    
    print(f"{threshold:<15} {metrics['precision']:<10.3f} {metrics['recall']:<10.3f} {metrics['f1_score']:<10.3f}")

# Plot threshold sensitivity
threshold_df = pd.DataFrame(threshold_results)

plt.figure(figsize=(10, 6))
plt.plot(threshold_df['threshold'], threshold_df['precision'], 'o-', label='Precision', linewidth=2)
plt.plot(threshold_df['threshold'], threshold_df['recall'], 's-', label='Recall', linewidth=2)
plt.plot(threshold_df['threshold'], threshold_df['f1_score'], '^-', label='F1-Score', linewidth=2)

plt.xlabel('IoU Threshold')
plt.ylabel('Score')
plt.title('Detection Performance vs IoU Threshold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.ylim(0, 1)
plt.show()

## 6. Cost-Sensitive Analysis

In semiconductor manufacturing, different types of errors have different costs. Let's analyze the economic impact of our detection system.

In [ ]:
# Cost sensitivity analysis
def cost_analysis(metrics, defect_costs, false_alarm_costs):
    """Analyze costs under different cost structures."""
    results = []
    
    for defect_cost in defect_costs:
        for false_alarm_cost in false_alarm_costs:
            estimated_loss = (
                metrics['false_negatives'] * defect_cost + 
                metrics['false_positives'] * false_alarm_cost
            )
            
            results.append({
                'defect_cost': defect_cost,
                'false_alarm_cost': false_alarm_cost,
                'total_loss': estimated_loss,
                'fn_loss': metrics['false_negatives'] * defect_cost,
                'fp_loss': metrics['false_positives'] * false_alarm_cost
            })
    
    return results

# Different cost scenarios
defect_costs = [500, 1000, 2000, 5000]  # Cost per missed defect
false_alarm_costs = [50, 100, 200, 500]  # Cost per false alarm

cost_results = cost_analysis(classical_metrics, defect_costs, false_alarm_costs)
cost_df = pd.DataFrame(cost_results)

# Create cost heatmap
cost_pivot = cost_df.pivot(index='defect_cost', columns='false_alarm_cost', values='total_loss')

plt.figure(figsize=(10, 8))
import matplotlib.pyplot as plt

# Create heatmap manually since seaborn might not be available
im = plt.imshow(cost_pivot.values, cmap='Reds', aspect='auto')

# Add labels
plt.xticks(range(len(cost_pivot.columns)), cost_pivot.columns)
plt.yticks(range(len(cost_pivot.index)), cost_pivot.index)
plt.xlabel('False Alarm Cost ($)')
plt.ylabel('Defect Miss Cost ($)')
plt.title('Total Estimated Loss Under Different Cost Structures')

# Add text annotations
for i in range(len(cost_pivot.index)):
    for j in range(len(cost_pivot.columns)):
        text = plt.text(j, i, f'${cost_pivot.iloc[i, j]:.0f}',
                       ha="center", va="center", color="black" if cost_pivot.iloc[i, j] < cost_pivot.values.max()/2 else "white")

plt.colorbar(im, label='Total Loss ($)')
plt.tight_layout()
plt.show()

print("\nCost Analysis Summary:")
print(f"Current False Negatives: {classical_metrics['false_negatives']}")
print(f"Current False Positives: {classical_metrics['false_positives']}")
print(f"\nMinimum Total Loss: ${cost_df['total_loss'].min():.0f}")
print(f"Maximum Total Loss: ${cost_df['total_loss'].max():.0f}")

## 7. Production Deployment Considerations

Let's explore practical considerations for deploying defect detection in production.

In [ ]:
# Performance benchmarking
def benchmark_inference_speed(pipeline, test_images, n_runs=5):
    """Benchmark inference speed."""
    times = []
    
    for run in range(n_runs):
        start_time = time.time()
        predictions = pipeline.predict(test_images)
        end_time = time.time()
        times.append(end_time - start_time)
    
    return {
        'mean_time': np.mean(times),
        'std_time': np.std(times),
        'images_per_second': len(test_images) / np.mean(times),
        'ms_per_image': (np.mean(times) / len(test_images)) * 1000
    }

# Benchmark classical detector
test_images = images[:3]
speed_results = benchmark_inference_speed(classical_pipeline, test_images)

print("Production Speed Benchmark:")
print(f"Mean inference time: {speed_results['mean_time']:.3f} ± {speed_results['std_time']:.3f} seconds")
print(f"Images per second: {speed_results['images_per_second']:.1f}")
print(f"Milliseconds per image: {speed_results['ms_per_image']:.1f} ms")

# Production throughput requirements
wafers_per_hour = [100, 500, 1000, 2000]
images_per_wafer = 10  # Assume 10 images per wafer

print("\nProduction Throughput Analysis:")
print(f"{'Wafers/Hour':<12} {'Images/Hour':<12} {'Required ms/Image':<18} {'Can Handle?':<12}")
print("-" * 60)

for wph in wafers_per_hour:
    images_per_hour = wph * images_per_wafer
    required_ms_per_image = (3600 * 1000) / images_per_hour  # ms per image
    can_handle = "✓" if speed_results['ms_per_image'] <= required_ms_per_image else "✗"
    
    print(f"{wph:<12} {images_per_hour:<12} {required_ms_per_image:<18.1f} {can_handle:<12}")

In [ ]:
# Model persistence and deployment
import tempfile
import os

# Save and reload model
with tempfile.NamedTemporaryFile(suffix='.joblib', delete=False) as f:
    model_path = f.name

try:
    # Save model
    classical_pipeline.save(Path(model_path))
    print(f"Model saved to: {model_path}")
    print(f"Model file size: {os.path.getsize(model_path)} bytes")
    
    # Load model
    loaded_pipeline = AdvancedDefectDetectionPipeline.load(Path(model_path))
    print(f"Model loaded successfully")
    print(f"Loaded backend: {loaded_pipeline.backend}")
    
    # Verify loaded model works
    test_predictions = loaded_pipeline.predict(test_images[:1])
    print(f"Loaded model predictions: {len(test_predictions[0])} detections")
    
    # Check metadata
    if loaded_pipeline.metadata:
        print(f"\nModel Metadata:")
        print(f"Trained at: {loaded_pipeline.metadata.trained_at}")
        print(f"Backend: {loaded_pipeline.metadata.backend}")
        print(f"Class names: {loaded_pipeline.metadata.class_names}")

finally:
    # Clean up
    if os.path.exists(model_path):
        os.unlink(model_path)

## 8. Summary and Next Steps

Let's summarize what we've learned and discuss next steps for production deployment.

In [ ]:
# Summary of results
print("Module 7.1 Summary: Advanced Defect Detection")
print("=" * 50)

print("\n1. Synthetic Data Generation:")
print(f"   - Generated {len(images)} synthetic wafer images")
print(f"   - Three defect types: scratches, particles, cracks")
print(f"   - Realistic wafer appearance with bounding box annotations")

print("\n2. Classical Detection Performance:")
print(f"   - Precision: {classical_metrics['precision']:.3f}")
print(f"   - Recall: {classical_metrics['recall']:.3f}")
print(f"   - mAP@0.5: {classical_metrics['map_50']:.3f}")
print(f"   - PWS: {classical_metrics['pws_percent']:.1f}%")
print(f"   - Inference speed: {speed_results['ms_per_image']:.1f} ms/image")

print("\n3. Backend Availability:")
for backend, available in backend_availability.items():
    status = "✓" if available else "✗"
    print(f"   {status} {backend.upper()}")

print("\n4. Key Insights:")
print("   - Classical methods provide fast, CPU-friendly baseline")
print("   - Deep learning offers higher accuracy but requires training data")
print("   - Cost-sensitive metrics essential for manufacturing context")
print("   - Model persistence enables production deployment")

print("\n5. Next Steps for Production:")
print("   - Collect real wafer defect data for training")
print("   - Implement A/B testing framework")
print("   - Set up continuous monitoring and retraining")
print("   - Optimize models for target hardware (CPU/GPU/Edge)")
print("   - Integrate with manufacturing execution systems")

print("\n6. Additional Resources:")
print("   - Fundamentals: 7.1-advanced-defect-detection-fundamentals.md")
print("   - Quick Reference: 7.1-advanced-defect-detection-quick-ref.md")
print("   - Production Pipeline: 7.1-advanced-defect-detection-pipeline.py")
print("   - Tests: test_advanced_detection_pipeline.py")

## 9. Hands-On Exercises

Try these exercises to deepen your understanding:

### Exercise 1: Parameter Tuning
Experiment with different classical detection parameters:
- Try `blur_kernel` values: 3, 5, 7, 9
- Try `threshold_value` values: 30, 50, 70, 90
- Find the combination that maximizes F1-score

### Exercise 2: Cost Optimization
Given your facility's actual costs:
- Missed defect cost: $2000
- False alarm cost: $150
- Determine optimal confidence threshold

### Exercise 3: Real Data Integration
If you have access to real wafer images:
- Replace synthetic data with real images
- Compare performance across different wafer types
- Analyze failure modes and edge cases

### Exercise 4: Production Deployment
Design a production system:
- Define SLA requirements (speed, accuracy)
- Choose appropriate backend
- Plan monitoring and alerting strategy

In [ ]:
# Exercise workspace - modify and run your experiments here

# Exercise 1: Parameter tuning example
print("Exercise 1: Parameter Tuning")
print("-" * 30)

best_f1 = 0
best_params = {}

for blur_kernel in [3, 5, 7]:
    for threshold_value in [30, 50, 70]:
        # Create pipeline with specific parameters
        test_pipeline = AdvancedDefectDetectionPipeline(
            backend='classical',
            blur_kernel=blur_kernel,
            threshold_value=threshold_value
        )
        
        # Train and evaluate
        test_pipeline.fit(images[:3], annotations[:3])
        metrics = test_pipeline.evaluate(images[3:6], annotations[3:6])
        
        print(f"Blur: {blur_kernel}, Threshold: {threshold_value} => F1: {metrics['f1_score']:.3f}")
        
        if metrics['f1_score'] > best_f1:
            best_f1 = metrics['f1_score']
            best_params = {'blur_kernel': blur_kernel, 'threshold_value': threshold_value}

print(f"\nBest parameters: {best_params}")
print(f"Best F1-score: {best_f1:.3f}")